<h1>Modelagem de dados sequenciais usando redes neurais recorrentes</h1>

<p align=center><img src=https://datascience.eu/wp-content/uploads/2020/05/image-513-1024x347.png></p>

Tivemos oportunidade de focar em redes neurais convolucionais (*CNNs*), de forma a cobrir os blocos de construção das arquiteturas *CNN* e como implementar *CNNs* profundas no *TensorFlow*. Por fim, você aprendeu a usar *CNNs* para classificação de imagens.

Aqui, exploraremos as redes neurais recorrentes (*RNNs*) e veremos sua aplicação na modelagem de dados sequenciais.

### Apresentando dados sequenciais
Vamos começar nossa discussão sobre *RNNs* observando a natureza dos dados sequenciais, que são mais comumente conhecidos como dados de sequência ou **sequências**. Vamos dar uma olhada nas propriedades únicas das sequências que as tornam diferentes de outros tipos de dados. Veremos então como podemos representar dados sequenciais e explorar as várias categorias de modelos para dados sequenciais, que são baseados na entrada e saída de um modelo. Isso nos ajudará a explorar a relação entre *RNNs* e sequências.

### Modelagem de dados sequenciais - a ordem importa

O que torna as sequências únicas, em comparação com outros tipos de dados, é que os elementos em uma sequência aparecem em uma determinada ordem e não são independentes uns dos outros. Algoritmos típicos de aprendizado de máquina para aprendizado supervisionado pressupõem que a entrada de dados é **independente e e distribuída de forma idêntica (IID)**, o que significa que os exemplos de treinamento são `mutuamente independentes` e têm a mesma distribuição subjacente.

Nesse sentido, com base na suposição de independência mútua, a ordem em que os exemplos de treinamento são dados ao modelo é **irrelevante**. Por exemplo, se tivermos uma amostra composta por n exemplos de treinamento, $\small x^{(1)}, x^{(2)},\cdots, x^{(n)} $, a ordem em que usamos os dados para treinar nosso algoritmo de aprendizado de máquina não importa. Um exemplo desse cenário seria o conjunto de dados Iris, muito conhecido. No conjunto de dados Iris, cada flor foi medida independentemente e as medidas de uma flor não influenciam as medidas de outra flor.

No entanto, essa suposição não é válida quando lidamos com sequências – por definição, **a ordem é importante**. Prever o valor de mercado de uma determinada ação seria um exemplo desse cenário. Por exemplo, suponha que temos uma amostra de n exemplos de treinamento, onde cada exemplo de treinamento representa o valor de mercado de uma determinada ação em um determinado dia. Se nossa tarefa é prever o valor do mercado de ações para os próximos três dias, faria sentido considerar os preços das ações anteriores em uma ordem de data para derivar tendências, em vez de utilizar esses exemplos de treinamento em uma ordem aleatória.

> #### Dados sequenciais versus dados de séries temporais
> Os dados de série temporal são um tipo especial de dados sequenciais, em que cada exemplo está associado a uma dimensão de tempo. Em dados de séries temporais, as amostras são coletadas em *timestamps* sucessivos e, portanto, a dimensão de tempo determina a ordem entre os pontos de dados. Por exemplo, preços de ações e registros de voz ou fala são dados de séries temporais.
>
> Por outro lado, nem todos os dados sequenciais têm a dimensão temporal, por exemplo, dados de texto ou sequências de DNA, onde os exemplos são ordenados, mas não se qualificam como dados de séries temporais. Como você verá, abordaremos alguns exemplos de Processamento de Linguagem Natural (NLP) e modelagem de texto que não são dados de série temporal, mas observe que as *RNNs* também podem ser usados para dados de série temporal.

### Representando sequências

Estabelecemos que a ordem entre os pontos de dados é importante em dados sequenciais, então precisamos encontrar uma maneira de aproveitar essas informações de pedido em um modelo de aprendizado de máquina. Ao longo das explicações, representaremos as sequências como $\small \left \langle x^{(1)},x^{(2)},\cdots, x^{(T)}  \right \rangle$ . Os índices sobrescritos indicam a ordem das instâncias e o comprimento da sequência é $\small T$. Para um exemplo sensato de sequências, considere dados de séries temporais, onde cada ponto de exemplo, $\small x^{(T)}$, pertence a um determinado tempo, $\small t$. A figura a seguir mostra um exemplo de dados de série temporal em que os recursos de entrada ($\small x$) e os rótulos de destino ($\small y$) seguem naturalmente a ordem de acordo com seu eixo de tempo; portanto, ambos os `x` e `y` são sequências:

![](imagens\sequencias.PNG)

Como já mencionamos, os modelos de rede neural padrão (*RN*) que abordamos até agora, como o *multilayer perceptron* (MLP) e as *CNNs* para dados de imagem, assumem que os exemplos de treinamento são independentes uns dos outros e, portanto, não incorporam **informação de ordenamento**. Podemos dizer que tais modelos não possuem **memória** de exemplos de treinamento vistos anteriormente. Por exemplo, as amostras são passadas pelas etapas de *feedforward* e *backpropagation* e os pesos são atualizados independentemente da ordem em que os exemplos de treinamento são processados. As *RNNs*, por outro lado, são projetadas para modelar sequências e são capazes de lembrar informações passadas e processar novos eventos de acordo, o que é uma clara vantagem ao trabalhar com dados de sequência.

### As diferentes categorias de modelagem de sequência

A modelagem de sequência tem muitas aplicações fascinantes, como tradução de idiomas (por exemplo, tradução de texto de inglês para alemão), legendas de imagens e geração de texto. No entanto, para escolher uma arquitetura e abordagem apropriadas, temos que entender e ser capazes de distinguir entre essas diferentes tarefas de modelagem de sequência. A figura a seguir, baseada nas explicações do excelente artigo The Unreasonable Effectiveness of Recurrent Neural Networks, de Andrej Karpathy (http://karpathy.github.io/2015/05/21/rnn-effectiveness/), resume a sequência mais comum tarefas de modelagem, que dependem das categorias de relacionamento de dados de entrada e saída:

![](imagens\modelagem_de_sequencia.PNG)

Vamos discutir as diferentes categorias de relacionamento entre dados de entrada e saída, que foram descritas na figura anterior, com mais detalhes. Se nem os dados de entrada nem de saída representam sequências, então estamos lidando com dados padrão e podemos simplesmente usar um perceptron multicamada (ou outro modelo de classificação abordado anteriormente) para modelar esses dados. No entanto, se a entrada ou a saída for uma sequência, a tarefa de modelagem provavelmente se enquadra em uma destas categorias:
* **Muitos para um**: Os dados de entrada são uma sequência, mas a saída é um vetor de tamanho fixo ou escalar, não uma sequência. Por exemplo, na análise de sentimentos, a entrada é baseada em texto (por exemplo, uma resenha de filme) e a saída é um rótulo de classe (por exemplo, um rótulo que indica se um revisor gostou do filme).

* **Um para muitos**: Os dados de entrada estão no formato padrão e não em sequência, mas a saída é uma sequência. Um exemplo dessa categoria é a legendagem de imagens — a entrada é uma imagem e a saída é uma frase em inglês que resume o conteúdo dessa imagem.

* **Muitos para muitos**: As matrizes de entrada e saída são sequências. Esta categoria pode ser dividida com base na sincronização da entrada e saída. Um exemplo de uma tarefa de modelagem sincronizada de muitos para muitos é a classificação de vídeo, onde cada quadro em um vídeo é rotulado. Um exemplo de uma tarefa de modelagem muitos-para-muitos *atrasada* seria traduzir uma linguagem para outra. Por exemplo, uma frase inteira em inglês deve ser lida e processada por uma máquina antes que sua tradução para o alemão seja produzida. 

Agora, depois de resumir as três grandes categorias de modelagem de sequência, podemos avançar para discutir a estrutura de uma RNN.

### RNNs para modelagem de sequências 
Nesta seção, antes de começarmos a implementar *RNNs* no *TensorFlow*, discutiremos os principais conceitos de *RNNs*. Começaremos examinando a estrutura típica de uma *RNN*, que inclui um componente recursivo para modelar dados de sequência. Em seguida, examinaremos como as ativações dos neurônios são computadas em uma *RNN* típica. Isso criará um contexto para discutirmos os desafios comuns no treinamento de *RNNs* e, em seguida, discutiremos soluções para esses desafios, como *LSTM* e unidades recorrentes fechadas (*GRUs*).

### Entendendo o mecanismo de loop *RNN*
Vamos começar com a arquitetura de uma *RNN*. A figura a seguir mostra uma *RN* *feedforward* padrão e um *RNN* lado a lado para comparação:

![](imagens\mecanismo_loop_rnn.PNG)

Ambas as redes têm apenas uma camada oculta. Nesta representação, as unidades não são exibidas, mas assumimos que a camada de entrada ($\small x$), a camada oculta ($\small h$) e a camada de saída ($\small o$) são vetores que contêm muitas unidades.

> ##### Determinando o tipo de saída de um RNN
> Essa arquitetura RNN genérica pode corresponder às duas categorias de modelagem de sequência em que a entrada é uma sequência. Normalmente, uma camada recorrente pode retornar uma sequência como saída, $\small \left \langle o^{(1)},o^{(2)},\cdots, o^{(T)}  \right \rangle$, ou simplesmente retornar a última saída (em $\small t = T$, ou seja, $\small o^{(T)}$). Assim, pode ser muitos para muitos ou muitos para um se, por exemplo, usarmos apenas o último elemento, $\small o^{(T)}$, como a saída final.
>
> Como você verá mais tarde, na *API TensorFlow Keras*, o comportamento de uma camada recorrente em relação ao retorno de uma sequência como saída ou simplesmente usar a última saída pode ser especificado definindo o argumento `return_sequences` como `True` ou `False`, respectivamente.

Em uma rede *feedforward* padrão, as informações fluem da entrada para a camada oculta e, em seguida, da camada oculta para a camada de saída. Por outro lado, em uma *RNN*, a camada oculta recebe sua entrada tanto da camada de entrada da etapa de tempo atual quanto da camada oculta da etapa de tempo anterior.

O fluxo de informações em etapas de tempo adjacentes na camada oculta permite que a rede tenha uma memória de eventos passados. Esse fluxo de informações geralmente é exibido como um *loop*, também conhecido como **recurrent edge** (borda recorrente) em notação de gráfico, que é como essa arquitetura *RNN* geral recebeu seu nome.

Semelhante aos *perceptrons* multicamadas, as *RNN*s podem consistir em várias camadas ocultas. Observe que é uma convenção comum se referir às *RNNs* com uma camada oculta como uma *RNN* de camada única, que não deve ser confundido com as RNs de camada única sem uma camada oculta, como *Adaline* ou *regressão logística*. A figura a seguir ilustra uma *RNN* com uma camada oculta (superior) e uma *RNN* com duas camadas ocultas (inferior):

![](imagens\rnn_oculta.PNG)

Para examinar a arquitetura das *RNNs* e o fluxo de informações, pode-se desdobrar uma representação compacta com uma aresta recorrente, que você pode ver na figura anterior.

Como sabemos, cada unidade oculta em uma *RN* padrão recebe apenas uma entrada – a pré-ativação de rede associada à camada de entrada. Em contraste, cada unidade oculta em uma *RNN* recebe dois conjuntos distintos de entrada – a pré-ativação da camada de entrada e a ativação da mesma camada oculta da etapa de tempo anterior, $\small t – 1$.

Na primeira etapa de tempo, $\small t = 0$, as unidades ocultas são inicializadas com zeros ou pequenos valores aleatórios. Então, em um passo de tempo em que $\small t > 0$, as unidades ocultas recebem sua entrada do ponto de dados no momento atual, $\small x^{(T)}$, e os valores anteriores das unidades ocultas em $\small t – 1$, indicados como $\small h^{(t-1)}$.

Da mesma forma, no caso de uma RNN multicamada, podemos resumir o fluxo de informações da seguinte forma:
* $\small layer$ = 1: Aqui, a camada oculta é representada como $h_1^{(t)}$ e recebe sua entrada do ponto de dados, $x^{(t)}$, e os valores ocultos na mesma camada, mas no passo de tempo anterior, $h_1^{(t - 1)}$.
* $\small layer$ = 2: A segunda camada oculta, $h_2^{(t)}$ , recebe suas entradas das saídas da camada abaixo na etapa de tempo atual $(o_1^{(t)})$ e seus próprios valores ocultos da etapa de tempo anterior, $h_2^{(t-1)}$.

Como, neste caso, cada camada recorrente deve receber uma sequência como entrada, todas as camadas recorrentes, exceto a última, devem retornar uma sequência como saída (ou seja, `return_sequences=True`). O comportamento da última camada recorrente depende do tipo de problema.

### Computando ativações em uma *RNN*
Agora que você entende a estrutura e o fluxo geral de informações em uma *RNN*, vamos ser mais específicos e calcular as ativações reais das camadas ocultas, bem como a camada de saída. Para simplificar, consideraremos apenas uma única camada oculta; no entanto, o mesmo conceito se aplica às *RNNs* multicamadas.

Cada aresta direcionada (as conexões entre caixas) na representação de uma *RNN* que acabamos de ver está associada a uma matriz de pesos. Esses pesos não dependem do tempo, $\small t$; portanto, eles são compartilhados ao longo do eixo do tempo. As diferentes matrizes de peso em uma *RNN* de camada única são as seguintes:
* $\small \textbf{W}_{xh}$: A matriz de peso entre a entrada, $\small x^{(t)}$, e a camada oculta, $\small \textbf{h}$
* $\small \textbf{W}_{hh}$: A matriz de peso associada à aresta recorrente
* $\small \textbf{W}_{ho}$: A matriz de peso entre a camada oculta e a camada de saída

Essas matrizes de peso são representadas na figura a seguir:

![](imagens\matriz_pesos.PNG)

Em certas implementações, você pode observar que as matrizes de peso, $\small \textbf{W}_{xh}$ e $\small \textbf{W}_{hh}$, são concatenadas a uma matriz combinada, $\small \textbf{W}_{h} = [\textbf{W}_{xh}; \textbf{W}_{hh}]$. Mais adiante, faremos uso dessa notação também.

A computação das ativações é muito semelhante aos *perceptrons* multicamadas padrão e outros tipos de *RNs* *feedforward*. Para a camada oculta, a entrada líquida, $\small \textbf{z}_ h$ (pré-ativação), é calculada através de uma combinação linear, ou seja, calculamos a soma das multiplicações das matrizes de peso com os vetores correspondentes e somamos a unidade de polarização:
$$
\small \textbf{z}_h^{(t)} = \text{W}_{xh} \textbf{x}^{(t)} + \textbf{W}_{hh}\textbf{h}^{(t-1)} + \textbf{b}_h
$$

Então, as ativações das unidades ocultas na etapa do tempo, $\small t$, são calculadas da seguinte forma:

$$
\textbf{h}^{(t)} = \phi_h (\textbf{z}_h^{(t)}) = \phi_h (\textbf{W}_{xh}\textbf{x}^{(t)} +  \textbf{W}_{hh}\textbf{x}^{(t-1)} + \textbf{b}_h)
$$
 
Aqui, $\small \textbf{b}_h$ é o vetor de polarização para as unidades ocultas e $\phi_h(\cdot)$ é a função de ativação da camada oculta.

Caso você queira usar a matriz de peso concatenada, $\small \textbf{W}_h = [\textbf{W}_{xh}; \textbf{W}_{hh}]$, a fórmula para calcular as unidades ocultas mudará da seguinte forma:

$$
\small \textbf{h}^{(t)} = \phi_h \left ([\textbf{W}_{xh}; \textbf{W}_{hh}] \begin{bmatrix}
\textbf{x}^{(t)}\\ 
\textbf{h}^{t-1}
\end{bmatrix}       
+ \textbf{b}_h          \right )
$$

Uma vez computadas as ativações das unidades ocultas no passo de tempo atual, serão computadas as ativações das unidades de saída, como segue:

$$
\small \textbf{o}^{(t)} = \phi_0 (\textbf{W}_{ho}\textbf{h}^{(t)} + \textbf{b}_0)
$$

Para ajudar a esclarecer ainda mais, a figura a seguir mostra o processo de cálculo dessas ativações com ambas as formulações:

![](imagens\matriz_sequencial.PNG)

> ##### Treinando RNNs usando retropropagação ao longo do tempo (BPTT)
> O algoritmo de aprendizado para RNNs foi introduzido em 1990: Backpropagation Through Time: What It Does and How to Do It (Paul Werbos, Proceedings of IEEE, 78(10): 1550-1560, 1990). A derivada dos gradientes pode ser um pouco complicada, mas a ideia básica é que a perda total, $\small L$, é a soma de todas as funções de perda nos momentos $\small t = 1$ a $\small t = T$:

$$
L = \sum^T_{t=1}L^{(t)}
$$

Como a perda no tempo $\small t$ depende das unidades ocultas em todos os passos de tempo anteriores $\small 1 : t$, o gradiente será calculado da seguinte forma:

$$
\small \dfrac{\partial L^{(t)}}{\partial \textbf{W}_{hh}} = \dfrac{\partial L^{(t)}}{\partial \textbf{o}^{(t)}} \times \dfrac{\partial \textbf{o}^{(t)}}{\partial \textbf{h}^{(t)}} \times \left ( \sum^t_{k=1}\dfrac{\partial \textbf{h}^{(t)}}{\partial \textbf{h}^{(k)}} \times \dfrac{\partial \textbf{h}^{(k)}}{\partial \textbf{W}_{hh}} \right )

$$

Aqui, $\small \dfrac{\partial \textbf{h}^{(t)}}{\partial \textbf{h}^{(k)}}$ é calculado como uma multiplicação de passos de tempo adjacentes:
$$
\small \dfrac{\partial \textbf{h}^{(t)}}{\partial \textbf{h}^{k}} = \prod ^t_{i=k+1} \dfrac{\partial \textbf{h}^{(i)}}{\partial \textbf{h}^{(i-1)}} 
$$

### Recorrência oculta versus recorrência de saída
Até agora, você viu redes recorrentes nas quais a camada oculta tem a propriedade recorrente. No entanto, observe que existe um modelo alternativo em que a conexão recorrente vem da camada de saída. Nesse caso, as ativações líquidas da camada de saída na etapa de tempo anterior, $\small \textbf{o}^{(t-1)}$, podem ser adicionadas de duas maneiras:
* Para a camada oculta no passo de tempo atual, $\small \textbf{h}^t$ (mostrado na figura a seguir como recorrência de saída para oculta)
* Para a camada de saída no passo de tempo atual, $\small \textbf{o}^{t}$ (mostrado na figura a seguir como recorrência de saída para saída)

![](imagens\recorrencia_saida.PNG)

Conforme mostrado na figura anterior, as diferenças entre essas arquiteturas podem ser vistas claramente nas conexões recorrentes. Seguindo nossa notação, os pesos associados à conexão recorrente serão denotados para a recorrência oculta para oculta por $\small \textbf{W}_{hh}$, para a recorrência saída para oculta por $\small \textbf{W}_{oh}$, e para a recorrência saída para saída por $\small \textbf{W}_{oo}$. Em alguns artigos da literatura, os pesos associados às conexões recorrentes também são denotados por $\small \textbf{W}_{rec}$.

Para ver como isso funciona na prática, vamos calcular manualmente a passagem direta para um desses tipos recorrentes. Usando a *API TensorFlow Keras*, uma camada recorrente pode ser definida por meio da *SimpleRNN*, que é semelhante à recorrência de saída para saída. No código a seguir, criaremos uma camada recorrente do *SimpleRNN* e executaremos uma passagem direta em uma sequência de entrada de comprimento 3 para calcular a saída. Também calcularemos manualmente a passagem direta e compararemos os resultados com os do *SimpleRNN*. Primeiro, vamos criar a camada e atribuir os pesos para nossos cálculos manuais:

In [1]:
import tensorflow as tf
tf.random.set_seed(1)

rnn_layer = tf.keras.layers.SimpleRNN(
    units=2, use_bias=True, 
    return_sequences=True)
rnn_layer.build(input_shape=(None, None, 5))

w_xh, w_oo, b_h = rnn_layer.weights

print('W_xh shape:', w_xh.shape)
print('W_oo shape:', w_oo.shape)
print('b_h shape:', b_h.shape)

W_xh shape: (5, 2)
W_oo shape: (2, 2)
b_h shape: (2,)


A forma de entrada para esta camada é `(None, None, 5)`, onde a primeira dimensão é a dimensão do lote (usando `None` para tamanho de lote variável), a segunda dimensão corresponde à sequência (usando `None` para o comprimento variável da sequência) e a última dimensão corresponde às características. Observe que definimos `return_sequences=True`, que, para uma sequência de entrada de comprimento 3, resultará na sequência de saída $\small \left \langle \textbf{o}^{(0)},\textbf{o}^{(1)}, \textbf{o}^{(2)} \right \rangle$. Caso contrário, ele retornaria apenas a saída final, $\textbf{o}^{(2)}$.

Agora, vamos chamar a passagem direta no `rnn_layer` e calcular manualmente as saídas em cada passo de tempo e compará-las:

In [2]:
x_seq = tf.convert_to_tensor(
    [[1.0]*5, [2.0]*5, [3.0]*5],
    dtype=tf.float32)


## output of SimepleRNN:
output = rnn_layer(tf.reshape(x_seq, shape=(1, 3, 5)))

## manually computing the output:
out_man = []
for t in range(len(x_seq)):
    xt = tf.reshape(x_seq[t], (1, 5))
    print('Time step {} =>'.format(t))
    print('   Input           :', xt.numpy())
    
    ht = tf.matmul(xt, w_xh) + b_h    
    print('   Hidden          :', ht.numpy())
    
    if t>0:
        prev_o = out_man[t-1]
    else:
        prev_o = tf.zeros(shape=(ht.shape))
        
    ot = ht + tf.matmul(prev_o, w_oo)
    ot = tf.math.tanh(ot)
    out_man.append(ot)
    print('   Output (manual) :', ot.numpy())
    print('   SimpleRNN output:'.format(t), output[0][t].numpy())
    print()

Time step 0 =>
   Input           : [[1. 1. 1. 1. 1.]]
   Hidden          : [[0.41464037 0.96012145]]
   Output (manual) : [[0.39240566 0.74433106]]
   SimpleRNN output: [0.39240566 0.74433106]

Time step 1 =>
   Input           : [[2. 2. 2. 2. 2.]]
   Hidden          : [[0.82928073 1.9202429 ]]
   Output (manual) : [[0.80116504 0.9912947 ]]
   SimpleRNN output: [0.80116504 0.9912947 ]

Time step 2 =>
   Input           : [[3. 3. 3. 3. 3.]]
   Hidden          : [[1.243921  2.8803642]]
   Output (manual) : [[0.95468265 0.9993069 ]]
   SimpleRNN output: [0.95468265 0.9993069 ]



Em nosso cálculo direto manual, usamos a função de ativação da tangente hiperbólica (tanh), uma vez que também é usada na `SimpleRNN` (a ativação padrão). Como você pode ver nos resultados impressos, as saídas dos cálculos de encaminhamento manual correspondem exatamente à saída da camada `SimpleRNN` em cada etapa de tempo. Espero que esta tarefa prática tenha esclarecido você sobre os mistérios das redes recorrentes.

### Os desafios de aprender interações de longo prazo

BPTT, que foi brevemente mencionado anteriormente, apresenta alguns novos desafios. Por causa do fator multiplicativo,$\small \dfrac{\partial \textbf{h}^{(t)}}{\partial \textbf{h}^{(k)}}$, ao calcular os gradientes de uma função de perda, surgem os chamados problemas de **gradiente de fuga e explosão**. Esses problemas são explicados pelos exemplos na figura a seguir, que mostra uma RNN com apenas uma unidade oculta para simplificar:

![](imagens\gradiente_fuga_explosao.PNG)


Basicamente, $\small \dfrac{\partial \textbf{h}^{(t)}}{\partial \textbf{h}^{(k)}}$ tem $\small t – k$ multiplicações; portanto, multiplicar o peso, $\small w$, por ele mesmo $\small t – k$ vezes resulta em um fator, $\small w^{t-w}$ . Como resultado, se $\small |w|< 1$, esse fator se torna muito pequeno quando $\small t – k$ é grande. Por outro lado, se o peso da aresta recorrente for $\small |w|> 1$ , então $w^{t-k}$ se torna muito grande quando $\small t – k$ é grande. Observe que $\small t – k$ grande se refere a dependências de longo alcance. Podemos ver que uma solução ingênua para evitar gradientes de fuga ou explosão pode ser alcançada garantindo $\small |w| = 1$.

Na prática, existem pelo menos três soluções para este problema:
* *Gradient clipping*
* *TBPTT*
* *LSTM*

Usando o *Gradient clipping*, especificamos um valor de corte ou limite para os gradientes e atribuímos esse valor de corte aos valores de gradiente que excedem esse valor. Em contraste, o *TBPTT* simplesmente limita ao número de passos de tempo que o sinal pode retropropagar após cada passagem para frente. Por exemplo, mesmo que a sequência tenha 100 elementos ou etapas, podemos retropropagar apenas as 20 etapas de tempo mais recentes.

Embora tanto o *Gradient clipping* quanto o *TBPTT* possam resolver o problema do **gradiente explosivo**, o truncamento limita o número de etapas que o gradiente pode efetivamente retornar e atualizar adequadamente os pesos. Por outro lado, o *LSTM*, projetado em 1997 por *Sepp Hochreiter* e *Jürgen Schmidhuber*, tem tido mais sucesso em eliminar e explodir problemas de gradiente ao modelar dependências de longo alcance por meio de
o uso de células de memória. Vamos discutir *LSTM* com mais detalhes.

### Células de memória de longo prazo

Como afirmado anteriormente, os *LSTMs* foram introduzidos pela primeira vez para superar o problema do gradiente de fuga. O bloco de construção de um *LSTM* é uma célula de memória, que essencialmente representa ou substitui a camada oculta de RNNs padrão.

Em cada célula de memória, há uma aresta recorrente que tem o peso desejável, $\small w = 1$, como discutimos, para superar os problemas de gradiente de fuga e explosão. Os valores associados a essa borda recorrente são chamados coletivamente de estado da célula. A estrutura desdobrada de uma célula *LSTM* moderna é mostrada na figura a seguir:

![](imagens\ltsm.PNG)

Observe que o estado da célula do passo de tempo anterior, $\small \textbf{C}^{(t-1)}$, é modificado para obter o estado da célula no passo de tempo atual, $\small \textbf{C}^{(t)}$, sem ser multiplicado diretamente por nenhum fator de peso. O fluxo de informação nesta célula de memória é controlado por várias unidades de computação (frequentemente chamadas de portas) que serão descritas aqui. Na figura anterior,$\small \bigodot$ refere-se ao produto **elemento-a-elemento** (multiplicação elemento-a-elemento) e $\small \bigoplus$ significa s**oma-elemento** (adição elemento-a-elemento). Além disso, $\small \textbf{x}^{(t)}$ refere-se aos dados de entrada no tempo $\small t$, e $\small \textbf{h}^{(t-1)}$ indica as unidades ocultas no tempo $\small t – 1$. Quatro caixas são indicadas com uma função de ativação, seja a função sigmóide ($\small \sigma$) ou $\small tanh$, e um conjunto de pesos; essas caixas aplicam uma combinação linear realizando multiplicações de matriz-vetor em suas entradas (que são $\small \textbf{h}^{(t-1)}$ e $\small \textbf{x}^{(t)}$). Essas unidades de computação com funções de ativação sigmóides, cujas unidades de saída são passadas por $\small \bigodot$, são chamadas de portas.

Em uma célula *LSTM*, existem três tipos diferentes de portas, que são conhecidas como **porta de esquecimento**, porta de entrada e porta de saída:

* A **porta de esquecimento** ($\small f_t$) permite que a célula de memória redefina o estado da célula sem crescer indefinidamente. Na verdade, o portão de esquecimento decide quais informações podem passar e quais informações devem ser suprimidas. Agora, $\small f_t$ é calculado da seguinte forma:

$$
\small f_t = \sigma(\textbf{W}_{xf}\textbf{x}^{(t)} + \textbf{W}_{hf}\textbf{h}^{(t-1)} + \textbf{b}_f)
$$


* A **porta de entrada** ($\small i_t$) e o **valor candidato** ($\breve{C}_t$) são responsáveis ​​por atualizar o estado da célula. Eles são calculados da seguinte forma:


**PAREI AQUI**






𝒊𝒊𝑡𝑡 = 𝜎𝜎(𝑾𝑾𝑥𝑥𝑥𝑥𝒙𝒙(𝑡𝑡) + 𝑾𝑾ℎ𝑥𝑥𝒉𝒉(𝑡𝑡−1) + 𝒃𝒃𝑑
𝑪𝑪̃𝑡𝑡 = tanh(𝑾𝑾𝑥𝑥𝑥𝑥 𝒙𝒙(𝑡𝑡) + 𝑾𝑾ℎ𝑥𝑥 𝒉𝒉(𝑡𝑡−1) + 𝒃𝒃𝑥𝑥
O estado da célula no tempo t é calculado da seguinte forma:
𝑪𝑪(𝑡𝑡) = (𝑪𝑪(𝑡𝑡−1) ⊙ 𝒇𝒇𝑡𝑡) ⨁ (𝒊𝒊𝑡𝑡 ⊙ 𝑪𝑪̃𝑡𝑡)
• A porta de saída (𝒐𝒐𝑡𝑡 ) decide como atualizar os valores das unidades ocultas:

𝒐𝒐𝑡𝑡 = 𝜎𝜎(𝑾𝑾𝑥𝑥𝑥𝑥 𝒙𝒙(𝑡𝑡) + 𝑾𝑾ℎ𝑥𝑥𝒉𝒉(𝑡𝑡𝑥) + 𝒃𝒃𝑡

Dado isso, as unidades ocultas no passo de tempo atual são calculadas da seguinte forma:
𝒉𝒉(𝑡𝑡) = 𝒐𝒐𝑡𝑡 ⊙ tanh(𝑪𝑪(𝑡𝑡))
A estrutura de uma célula LSTM e seus cálculos subjacentes podem parecer muito complexos e difíceis de implementar. No entanto, a boa notícia é que o TensorFlow já implementou tudo em funções otimizadas de wrapper, o que nos permite definir nossas células LSTM de maneira fácil e eficiente. Aplicaremos RNNs e LSTMs a conjuntos de dados do mundo real posteriormente neste capítulo.

Outros modelos RNN avançados

LSTMs fornecem uma abordagem básica para modelar dependências de longo alcance em sequências. No entanto, é importante notar que existem muitas variações de LSTMs descritas na literatura (An Empirical Exploration of Recurrent Network Architectures, Rafal Jozefowicz, Wojciech Zaremba e Ilya Sutskever, Proceedings of ICML, 2342-2350, 2015). Também merece destaque uma abordagem mais recente, Gated Recurrent Unit (GRU), proposta em 2014. As GRUs possuem uma arquitetura mais simples que as LSTMs; portanto, eles são computacionalmente mais eficientes, enquanto seu desempenho em algumas tarefas, como modelagem de música polifônica, é comparável aos LSTMs. Se você estiver interessado em aprender mais sobre essas arquiteturas RNN modernas, consulte o artigo Empirical Evaluation of Gated Recurrent Neural Networks on Sequence Modeling, de Junyoung Chung e outros, 014
(https://arxiv.org/pdf/1412.3555v1.pdf).